In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Input, Dense
from mpl_toolkits import mplot3d

data_in, data_out = [], []

x = np.linspace(start=0.01, stop=10, num=50)
e = np.linspace(start=0.01, stop=0.3, num=50)

for xi in x:
    for ei in e:
        data_in.append([xi**2+xi+ei, 
                        np.sin(xi-np.pi/4)+ei,
                        np.log(np.abs(xi)+ei),
                        -xi**3+ei,
                        -(xi/4)+ei,
                        -xi+ei])
        data_out.append(np.abs(xi)+ei)

X = np.array(data_in)
y = np.array(data_out).reshape((2500, 1))

print(X)
print()
print(y)

[[ 2.01000000e-02 -6.90000476e-01 -3.91202301e+00  9.99900000e-03
   7.50000000e-03  0.00000000e+00]
 [ 2.60183673e-02 -6.84082109e-01 -3.65280340e+00  1.59173673e-02
   1.34183673e-02  5.91836735e-03]
 [ 3.19367347e-02 -6.78163741e-01 -3.44713448e+00  2.18357347e-02
   1.93367347e-02  1.18367347e-02]
 ...
 [ 1.10288163e+02  4.96795417e-01  2.33099404e+00 -9.99711837e+02
  -2.21183673e+00 -9.71183673e+00]
 [ 1.10294082e+02  5.02713784e-01  2.33156913e+00 -9.99705918e+02
  -2.20591837e+00 -9.70591837e+00]
 [ 1.10300000e+02  5.08632151e-01  2.33214390e+00 -9.99700000e+02
  -2.20000000e+00 -9.70000000e+00]]

[[ 0.02      ]
 [ 0.02591837]
 [ 0.03183673]
 ...
 [10.28816327]
 [10.29408163]
 [10.3       ]]


In [ ]:
input_layer = Input(shape=(6,))

encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(8, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(6, activation='linear')(decoded)

regress = Dense(4, activation='relu')(encoded)
regress = Dense(1, activation='linear')(regress)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X, X, epochs=50, batch_size=32, verbose=0)

encoder_model = Model(inputs=input_layer, outputs=encoded)
decoder_input = Input(shape=(8,))
decoder_output = autoencoder.layers[-2](decoder_input)
decoder_output = autoencoder.layers[-1](decoder_output)
decoder_model = Model(inputs=decoder_input, outputs=decoder_output)
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')
del encoder_model, decoder_model

regression_model = Model(inputs=input_layer, outputs=regress)
regression_model.compile(optimizer='adam', loss='mse')
regression_model.fit(X, y, epochs=50, batch_size=32, verbose=0)
regression_model.save('regression_model.h5')
del regression_model

In [ ]:
encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')
regression_model = load_model('regression_model.h5')

X_test = np.random.normal(loc=0, scale=10, size=(1000, 6))

encoded_data = encoder_model.predict(X_test)
decoded_data = decoder_model.predict(encoded_data)

y_pred = regression_model.predict(X)
y_true = np.abs(y)

79/79 [==============================] - 0s 1ms/step


In [ ]:
x1 = X[:, 0].reshape((2500, 1)).tolist()
x2 = X[:, 1].reshape((2500, 1)).tolist()
x3 = X[:, 3].reshape((2500, 1)).tolist()
x4 = X[:, 4].reshape((2500, 1)).tolist()
x5 = X[:, 4].reshape((2500, 1)).tolist()
x6 = X[:, 5].reshape((2500, 1)).tolist()

df1 = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x5': x5, 'x6': x6, 'y': y.tolist()})
df1.to_csv('generated_data.csv', index=False)
df1

,x1,x2,x3,x4,x5,x6,y
0,[0.0201],[-0.6900004761807905],[0.009999000000000001],[0.0075],[0.0075],[0.0],[0.02]
1,[0.026018367346938777],[-0.6840821088338518],[0.015917367346938775],[0.013418367346938775],[0.013418367346938775],[0.005918367346938775],[0.025918367346938774]
2,[0.03193673469387755],[-0.6781637414869129],[0.021835734693877548],[0.01933673469387755],[0.01933673469387755],[0.011836734693877549],[0.03183673469387755]
3,[0.037855102040816324],[-0.6722453741399742],[0.027754102040816325],[0.025255102040816328],[0.025255102040816328],[0.017755102040816324],[0.03775510204081633]
4,[0.0437734693877551],[-0.6663270067930355],[0.0336724693877551],[0.031173469387755105],[0.031173469387755105],[0.0236734693877551],[0.043673469387755105]
...,...,...,...,...,...,...,...
2495,[110.27632653061224],[0.4849586821042585],[-999.7236734693878],[-2.223673469387755],[-2.223673469387755],[-9.723673469387755],[10.276326530612245]
2496,[110.28224489795919],[0.49087704945119726],[-999.7177551020408],[-2.217755102040816],[-2.217755102040816],[-9.717755102040817],[10.282244897959183]
2497,[110.28816326530612],[0.4967954167981361],[-999.7118367346939],[-2.2118367346938776],[-2.2118367346938776],[-9.711836734693877],[10.288163265306123]
2498,[110.29408163265306],[0.5027137841450748],[-999.705918367347],[-2.2059183673469387],[-2.2059183673469387],[-9.70591836734694],[10.29408163265306]


In [ ]:
df3 = pd.DataFrame({'y_true': y_true.reshape((2500, )), 'y_pred': y_pred.reshape((2500, ))})
df3.to_csv('regression_results.csv', index=False)
df3

,y_true,y_pred
0,0.020000,0.023710
1,0.025918,0.029494
2,0.031837,0.036522
3,0.037755,0.042473
4,0.043673,0.047663
...,...,...
2495,10.276327,9.912714
2496,10.282245,9.914143
2497,10.288163,9.915588
2498,10.294082,9.917048
